# Football Scheduler

Here we implement the model descripted at [[1]](#References) and run some test with real teams.

## Imports

In [1]:
import numpy as np
import random
from football_scheduler_model import FootballSchedulerModel, SymetricScheme
from typing import Callable, Tuple

In [2]:
countries = ["ARG", "BOL", "BRA", "CHI", "COL", "PAR", "PER", "URU", "VEN"]
top_countries = ["ARG", "BRA"]
assert(set(top_countries).issubset(set(countries)))

In [3]:
n = len(countries)
k = 2 * n

In [4]:
def raffle(countries: list[str]) -> Tuple[Callable[[str], int], Callable[[int], str]]:
    countries_copy = countries.copy()
    random.shuffle(countries_copy)
    return lambda c: countries_copy.index(c), lambda i:countries_copy[i]

index_of, country_of = raffle(countries)

In [5]:
model_mirrored = FootballSchedulerModel(n, k, SymetricScheme.MIRRORED)
model_mirrored.optimize()

KeyError: (0, 9, 0)

## References:
- [1] G. Durán, E. Mijangos, and M. Frisk, “Scheduling the South American qualifiers to the 2018 FIFA World Cup by integer programming,” European Journal of Operational Research, vol. 262, no. 3, pp. 1035–1048, 2017.
- [2] https://pyscipopt.readthedocs.io/en/latest/index.html